# Подробное руководство по ноутбуку

Этот ноутбук показывает полный цикл подготовки признаков и обучения модели для прогноза относительного отклонения потребления горячей воды. Ниже приводится поэтапное описание, чтобы было понятно, что делает каждый блок и в каком порядке лучше их запускать.

1. **Установка зависимостей.** В первой ячейке мы ставим `catboost` и `optuna`. Выполните её только один раз в текущей среде — повторный запуск можно пропустить, если библиотеки уже установлены.
2. **Импорт библиотек и вспомогательных функций.** Вторая ячейка подключает стандартные пакеты, а также функции из `training_pipeline.data_loader`, которые загружают телеметрию и погодные признаки.
3. **Загрузка подготовленных датасетов.** Третья ячейка вызывает `prepare_model_frame()` и возвращает готовый датафрейм, в котором телеметрия объединена с погодой по дате и административному округу.
4. **Фиче-инжиниринг и целевая переменная.** Четвёртая ячейка добавляет показатель относительного отклонения `(odpu_hot - itp_cold) / itp_cold`, бинарную метку аномалии (>10%), календарные признаки и скользящие агрегаты, необходимые модели. Перед запуском убедитесь, что предыдущая ячейка выполнена.
5. **Разбиение выборки.** Пятая ячейка формирует обучающую и тестовую выборки, выделяет список категориальных признаков и создает объекты `Pool` для CatBoost.
6. **Гипертюнинг с Optuna.** Шестая ячейка запускает исследование гиперпараметров. Выполняется относительно долго: можно регулировать количество попыток через параметр `n_trials`.
7. **Финальное обучение.** Седьмая ячейка переобучает модель на объединённой обучающей выборке с лучшими найденными параметрами и строит прогноз для тестовой части.
8. **Подсчёт метрик и анализ аномалий.** Восьмая ячейка рассчитывает RMSE, MAE, R^2, точность по метке аномалии и формирует сводную таблицу фактов/прогнозов.
9. **Сохранение артефактов.** Девятая ячейка записывает модель (`models/catboost_hot_water.cbm`) и файл с метриками (`models/training_metrics.json`), чтобы фронтенд мог их подхватывать.

Каждая последующая ячейка предполагает успешное выполнение всех предыдущих. Если вы переоткрыли ноутбук, начните с импорта и повторно прогоните шаги 3–9.

### Прогноз отклонений показаний ГВС

Ниже представлен обновлённый пайплайн подготовки данных и обучения CatBoost на синтетических таблицах телеметрии и погодных факторов.

In [ ]:
!pip install -q catboost optuna

In [ ]:
# Parameters injected by papermill
telemetry_path = "data/telemetry.parquet"
weather_path = "data/weather_features.parquet"
run_id = "local"
output_model_path = f"models/catboost_{run_id}.cbm"
metrics_report_path = f"reports/training_metrics_{run_id}.json"
start_date = None
end_date = None


In [ ]:
import json
from pathlib import Path

import numpy as np
import optuna
import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import f1_score, mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

from training_pipeline import prepare_model_frame


In [ ]:
# Загружаем телеметрию и погодные признаки

def _load_dataset(path: str):
    data_path = Path(path)
    if data_path.suffix == ".parquet":
        return pd.read_parquet(data_path)
    if data_path.suffix == ".csv":
        return pd.read_csv(data_path)
    raise ValueError(f"Unsupported dataset format: {data_path}")

telemetry_df = _load_dataset(telemetry_path)
weather_df = _load_dataset(weather_path)
model_df = prepare_model_frame(telemetry=telemetry_df, weather=weather_df)

if start_date:
    model_df = model_df[model_df["date"] >= pd.to_datetime(start_date)]
if end_date:
    model_df = model_df[model_df["date"] <= pd.to_datetime(end_date)]

model_df = model_df.reset_index(drop=True)
model_df.head()

In [ ]:
# Фиче-инжиниринг: относительное отклонение, лаги, скользящие средние и категориальные признаки
model_df = model_df.copy()
model_df['target_deviation'] = (model_df['odpu_hot'] - model_df['itp_cold']) / model_df['itp_cold']
model_df['anomaly_label'] = (model_df['target_deviation'] > 0.10).astype(int)
model_df['month'] = model_df['date'].dt.month
model_df['dayofyear'] = model_df['date'].dt.dayofyear
model_df['days_from_start'] = (model_df['date'] - model_df['date'].min()).dt.days
telemetry_cols = ['odpu_hot', 'itp_hot', 'itp_cold', 'odpu_cold', 'hot_water_consumption', 'cold_water_consumption']
for col in telemetry_cols:
    model_df[f'{col}_lag1'] = model_df.groupby('mkd_id')[col].shift(1)
    model_df[f'{col}_ma3'] = model_df.groupby('mkd_id')[col].transform(
        lambda s: s.rolling(window=3, min_periods=1).mean()
    )
model_df = model_df.dropna().reset_index(drop=True)
model_df.head()

In [ ]:
# Разделяем выборку на train/test
feature_cols = [
    'mkd_id', 'mkd_address', 'district', 'itp_id', 'itp_address',
    'mkd_lat', 'mkd_lon', 'itp_lat', 'itp_lon',
    'odpu_hot', 'itp_hot', 'itp_cold', 'odpu_cold',
    'hot_water_consumption', 'cold_water_consumption',
    'avg_temp_c', 'min_temp_c', 'max_temp_c',
    'heating_degree_days', 'precipitation_mm', 'relative_humidity',
    'cloudiness_hours', 'wind_speed_ms', 'wind_load_subzero',
    'operational_period',
    'month', 'dayofyear', 'days_from_start',
] + [col for col in model_df.columns if col.endswith('_lag1') or col.endswith('_ma3')]
feature_cols = list(dict.fromkeys(feature_cols))  # удаляем дубликаты
target_col = 'target_deviation'
categorical_features = ['mkd_id', 'mkd_address', 'district', 'itp_id', 'itp_address', 'operational_period']
cat_feature_indices = [feature_cols.index(col) for col in categorical_features]
X = model_df[feature_cols]
y = model_df[target_col]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)
len(X_train), len(X_test)

In [ ]:
# Подбор гиперпараметров CatBoostRegressor с Optuna
def objective(trial: optuna.Trial) -> float:
    params = {
        'depth': trial.suggest_int('depth', 4, 8),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-2, 10.0, log=True),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 5.0),
        'border_count': trial.suggest_int('border_count', 64, 254),
    }
    X_tr, X_val, y_tr, y_val = train_test_split(
        X_train, y_train, test_size=0.25, random_state=42, shuffle=True
    )
    train_pool = Pool(X_tr, y_tr, cat_features=cat_feature_indices)
    valid_pool = Pool(X_val, y_val, cat_features=cat_feature_indices)
    model = CatBoostRegressor(
        iterations=300,
        loss_function='RMSE',
        random_seed=42,
        verbose=False,
        **params,
    )
    model.fit(train_pool, eval_set=valid_pool, use_best_model=True)
    preds = model.predict(valid_pool)
    rmse = mean_squared_error(y_val, preds) ** 0.5
    return rmse

study = optuna.create_study(direction='minimize', study_name='catboost-hot-water')
study.optimize(objective, n_trials=10, timeout=300)
study.best_params

In [ ]:
# Обучаем финальную модель на объединённой train-выборке
best_params = study.best_params
train_pool = Pool(X_train, y_train, cat_features=cat_feature_indices)
test_pool = Pool(X_test, y_test, cat_features=cat_feature_indices)
model = CatBoostRegressor(
    iterations=500,
    loss_function='RMSE',
    random_seed=42,
    verbose=False,
    **best_params,
)
model.fit(train_pool, eval_set=test_pool, use_best_model=True)
test_predictions = model.predict(test_pool)
test_predictions[:5]

In [ ]:
# Считаем метрики качества и бинарные метки аномалий
rmse = mean_squared_error(y_test, test_predictions) ** 0.5
mae = mean_absolute_error(y_test, test_predictions)
r2 = r2_score(y_test, test_predictions)
true_anomaly = (y_test > 0.10).astype(int)
pred_anomaly = (test_predictions > 0.10).astype(int)
f1 = f1_score(true_anomaly, pred_anomaly)
metrics = {
    'rmse': rmse,
    'mae': mae,
    'r2': r2,
    'f1_anomaly': f1,
    'best_params': best_params,
}
metrics

In [ ]:
# Сохраняем модель и метрики для фронтенда
model_path = Path(output_model_path)
metrics_path = Path(metrics_report_path)
model_path.parent.mkdir(parents=True, exist_ok=True)
metrics_path.parent.mkdir(parents=True, exist_ok=True)

model.save_model(model_path)
with metrics_path.open('w', encoding='utf-8') as fp:
    json.dump(metrics, fp, ensure_ascii=False, indent=2)

model_path, metrics_path